# Table for constructing a star schema


In [47]:
import pandas as pd
import numpy as np
import ast

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', None)

In [48]:
df = pd.read_csv("../step2_data cleaning and preprocessing/cleaned_df.csv")
print(df.head(10))

      id                                 title                       company       type                                industry           created_at  is_active sub_region      region                      skills_extracted salary_bucket accredited_label
0  21070                 Deafblind Coordinator           Blind Low Vision NZ  Full time        Community Services & Development  2024-08-24 20:55:36          0    Unknown    Auckland    ['Microsoft Office', 'Salesforce']    Negotiable   Not Accredited
1  21840  Site supervisor for structural steel  Grayson Engineering 2015 Ltd  Full time                            Construction  2024-08-24 20:56:50          0    Unknown    Auckland                                    []    Negotiable   Not Accredited
2  22213      Project manager structural steel  Grayson Engineering 2015 Ltd  Full time                            Construction  2024-08-24 20:57:25          0    Unknown    Auckland                           ['Outlook']    Negotiable   Not Acc

## Dimension Tables

### 1. dim_skills & fact_job_skills(helper fact)

#### - dim_skills

In [49]:
df["skills_extracted"] = df["skills_extracted"].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else [])
df["skills_extracted"] = df["skills_extracted"].apply(lambda x: [s.strip().title() for s in x if isinstance(s, str)])

all_skills = sorted(set(skill for sublist in df["skills_extracted"] for skill in sublist if skill))
dim_skills = pd.DataFrame({
    "skill_id": range(1, len(all_skills) + 1),
    "skill_name": all_skills
})
print(dim_skills.head(30))


    skill_id    skill_name
0          1       Airflow
1          2       Angular
2          3  Apache Spark
3          4           Aws
4          5         Azure
5          6      Bigquery
6          7     Bootstrap
7          8             C
8          9           C++
9         10          Chai
10        11    Confluence
11        12           Css
12        13       Cypress
13        14    Databricks
14        15        Docker
15        16  Dynamics 365
16        17       Eclipse
17        18         Excel
18        19      Firebase
19        20           Gcp
20        21           Git
21        22        Github
22        23  Google Cloud
23        24       Graphql
24        25          Html
25        26   Informatica
26        27      Intellij
27        28          Java
28        29    Javascript
29        30       Jenkins


In [50]:
dim_skills.to_csv("tables/dim_skills.csv", index=False)

#### - fact_job_skills

In [51]:
bridge_records = []
for idx, row in df.iterrows():
    for skill in row["skills_extracted"]:
        skill_id = dim_skills.loc[dim_skills["skill_name"] == skill, "skill_id"].values[0]
        bridge_records.append({
            "id": row["id"],
            "skill_id": skill_id
        })

fact_job_skills = pd.DataFrame(bridge_records)
print(fact_job_skills.head(40))

        id  skill_id
0    21070        38
1    21070        62
2    22213        46
3    29815        70
4    29830        70
5    29830        67
6    29830         8
7    29830        60
8    30198        38
9    36232        18
10   63827        38
11   65498        18
12   65839        30
13   65839        34
14   65839        41
15   65839        61
16   65839        50
17   65839        15
18   65839        54
19   65839        21
20   65839        70
21   65839        36
22   65920        29
23   65920         8
24   65935        29
25   65935         8
26   66007        29
27   66007         8
28   66027        29
29   66027         8
30   66780        70
31   66784        70
32   66784        67
33   66784         8
34   66784        60
35  104471         5
36  137457        18
37  138230        38
38  138230        18
39  138230        46


In [52]:
dim_skills.to_csv('tables/fact_job_skills.csv', index=False)

### 2. dim_locations

In [53]:
dim_locations = df[['sub_region', 'region']].drop_duplicates().reset_index(drop=True)
dim_locations['location_id'] = np.arange(1, len(dim_locations) + 1)
cols = ['location_id'] + [col for col in dim_locations.columns if col != 'location_id']
dim_locations = dim_locations[cols]
print(dim_locations.head(20))

    location_id    sub_region         region
0             1       Unknown       Auckland
1             2       Unknown     Canterbury
2             3       Unknown          Otago
3             4       Unknown     Wellington
4             5       Unknown       Taranaki
5             6       Unknown       Manawatu
6             7       Unknown        Waikato
7             8       Unknown      Northland
8             9       Unknown  Bay of Plenty
9            10       Unknown      Southland
10           11       Unknown       Gisborne
11           12       Unknown     Hawkes Bay
12           13       Unknown         Tasman
13           14       Unknown     West Coast
14           15       Unknown    Marlborough
15           16        Hornby     Canterbury
16           17      Onehunga       Auckland
17           18       Porirua     Wellington
18           19  Auckland CBD       Auckland
19           20  Christchurch     Canterbury


In [54]:
dim_locations.to_csv('tables/dim_locations.csv', index=False)

### 3. dim_titles

In [55]:
dim_titles = df[['title']].drop_duplicates().reset_index(drop=True)
dim_titles['title_id'] = np.arange(1, len(dim_titles) + 1)
cols = ['title_id'] + [col for col in dim_titles.columns if col != 'title_id']
dim_titles = dim_titles[cols]
print(dim_titles.head(10))

   title_id                                 title
0         1                 Deafblind Coordinator
1         2  Site supervisor for structural steel
2         3      Project manager structural steel
3         4          Medical Device Kit Assembler
4         5                Holiday Park Assistant
5         6             Venue Manager - Bakehouse
6         7                 Senior Analyst Tester
7         8                        Analyst Tester
8         9                   Construction Lawyer
9        10               Legal Executive Estates


In [56]:
dim_titles.to_csv('tables/dim_titles.csv', index=False)

### 4. dim_companies

In [57]:
dim_companies = df[['company']].drop_duplicates().reset_index(drop=True)
dim_companies['company_id'] = np.arange(1, len(dim_companies) + 1)
cols = ['company_id'] + [col for col in dim_companies.columns if col != 'company_id']
dim_companies = dim_companies[cols]
print(dim_companies.head(10))

   company_id                       company
0           1           Blind Low Vision NZ
1           2  Grayson Engineering 2015 Ltd
2           3                       Stryker
3           4       Lakes Edge Holiday Park
4           5      Ayrburn Precinct Limited
5           6     FNZ Services (NZ) Limited
6           7         Forte Recruitment Ltd
7           8         Nicholsons Solicitors
8           9            Star Personnel Ltd
9          10         McAlpine Hussmann Ltd


In [58]:
dim_companies.to_csv('tables/dim_companies.csv', index=False)

### 5. dim_job_type

In [59]:
dim_job_type = df[['type']].drop_duplicates().reset_index(drop=True)
dim_job_type['job_type_id'] = np.arange(1, len(dim_job_type) + 1)
cols = ['job_type_id'] + [col for col in dim_job_type.columns if col != 'job_type_id']
dim_job_type = dim_job_type[cols]
print(dim_job_type.head(10))

   job_type_id             type
0            1        Full time
1            2        Part time
2            3    Contract/Temp
3            4  Casual/Vacation


In [60]:
dim_job_type.to_csv('tables/dim_job_type.csv', index=False)

### 6. dim_industries

In [61]:
dim_industries = df[['industry']].drop_duplicates().reset_index(drop=True)
dim_industries['industry_id'] = np.arange(1, len(dim_industries) + 1)
cols = ['industry_id'] + [col for col in dim_industries.columns if col != 'industry_id']
dim_industries = dim_industries[cols]
print(dim_industries.head(10))

   industry_id                                industry
0            1        Community Services & Development
1            2                            Construction
2            3                    Healthcare & Medical
3            4                   Hospitality & Tourism
4            5  Information & Communication Technology
5            6                                   Legal
6            7    Manufacturing, Transport & Logistics
7            8                       Trades & Services
8            9           Human Resources & Recruitment
9           10            Banking & Financial Services


In [62]:
dim_industries.to_csv('tables/dim_industries.csv', index=False)

### 7. dim_datetime

In [63]:
df['created_at'] = pd.to_datetime(df['created_at'], errors='coerce')

# 生成日期范围
start_date = df['created_at'].min().normalize()
end_date = pd.Timestamp.today().normalize()
date_range = pd.date_range(start=start_date, end=end_date, freq='D')

# 构建维表
dim_datetime = pd.DataFrame({'date': date_range})
dim_datetime['year'] = dim_datetime['date'].dt.year
dim_datetime['month'] = dim_datetime['date'].dt.month
dim_datetime['day'] = dim_datetime['date'].dt.day
dim_datetime['quarter'] = dim_datetime['date'].dt.quarter
dim_datetime['week'] = dim_datetime['date'].dt.isocalendar().week
dim_datetime['weekday'] = dim_datetime['date'].dt.weekday + 1  # 1=Monday
dim_datetime['datetime_id'] = dim_datetime['date'].dt.strftime('%Y%m%d').astype(int)

# 调整列顺序
cols = ['datetime_id', 'date', 'year', 'quarter', 'month', 'week', 'day', 'weekday']
dim_datetime = dim_datetime[cols]

print(dim_datetime.head(10))

   datetime_id       date  year  quarter  month  week  day  weekday
0     20240824 2024-08-24  2024        3      8    34   24        6
1     20240825 2024-08-25  2024        3      8    34   25        7
2     20240826 2024-08-26  2024        3      8    35   26        1
3     20240827 2024-08-27  2024        3      8    35   27        2
4     20240828 2024-08-28  2024        3      8    35   28        3
5     20240829 2024-08-29  2024        3      8    35   29        4
6     20240830 2024-08-30  2024        3      8    35   30        5
7     20240831 2024-08-31  2024        3      8    35   31        6
8     20240901 2024-09-01  2024        3      9    35    1        7
9     20240902 2024-09-02  2024        3      9    36    2        1


In [64]:
dim_datetime.to_csv('tables/dim_datetime.csv', index=False)

### 8. dim_salary

In [65]:
dim_salary = df[['salary_bucket']].drop_duplicates().reset_index(drop=True)
dim_salary['salary_id'] = np.arange(1, len(dim_salary) + 1)
cols = ['salary_id'] + [col for col in dim_salary.columns if col != 'salary_id']
dim_salary = dim_salary[cols]
print(dim_salary.head(10))

   salary_id salary_bucket
0          1    Negotiable
1          2         Other
2          3      50k-100k
3          4         100k+
4          5         0-50k
5          6   Competitive


In [66]:
dim_salary.to_csv('tables/dim_salary.csv', index=False)

### 9. dim_accredited

In [67]:
dim_accredited = df[['accredited_label']].drop_duplicates().reset_index(drop=True)
dim_accredited['accredited_id'] = np.arange(1, len(dim_accredited) + 1)
cols = ['accredited_id'] + [col for col in dim_accredited.columns if col != 'accredited_id']
dim_accredited = dim_accredited[cols]
print(dim_accredited.head(10))

   accredited_id accredited_label
0              1   Not Accredited
1              2       Accredited
2              3          Unknown


In [68]:
dim_accredited.to_csv('tables/dim_accredited.csv', index=False)

### 10. dim_is_active

In [69]:
dim_is_active = pd.DataFrame({
    'active_id': [0, 1],
    'is_active': ['Not Active', 'Active']
})
print(dim_is_active.head())


   active_id   is_active
0          0  Not Active
1          1      Active


In [70]:
df.rename(columns={'is_active': 'active_id'}, inplace=True)
print(df.head(10))

      id                                 title                       company       type                                industry          created_at  active_id sub_region      region                skills_extracted salary_bucket accredited_label
0  21070                 Deafblind Coordinator           Blind Low Vision NZ  Full time        Community Services & Development 2024-08-24 20:55:36          0    Unknown    Auckland  [Microsoft Office, Salesforce]    Negotiable   Not Accredited
1  21840  Site supervisor for structural steel  Grayson Engineering 2015 Ltd  Full time                            Construction 2024-08-24 20:56:50          0    Unknown    Auckland                              []    Negotiable   Not Accredited
2  22213      Project manager structural steel  Grayson Engineering 2015 Ltd  Full time                            Construction 2024-08-24 20:57:25          0    Unknown    Auckland                       [Outlook]    Negotiable   Not Accredited
3  26397          Me

In [71]:
dim_is_active.to_csv('tables/dim_is_active.csv', index=False)

## Fact table

### fact_jobs

In [75]:
df['created_at'] = pd.to_datetime(df['created_at'], errors='coerce')
df['datetime_id'] = df['created_at'].dt.strftime('%Y%m%d').astype(int)

# 合并各维度ID
fact_jobs = df.merge(dim_titles, on='title', how='left') \
    .merge(dim_companies, on='company', how='left') \
    .merge(dim_locations, on=['sub_region', 'region'], how='left') \
    .merge(dim_job_type, on='type', how='left') \
    .merge(dim_industries, on='industry', how='left') \
    .merge(dim_salary, on='salary_bucket', how='left') \
    .merge(dim_accredited, on='accredited_label', how='left') \
    .merge(dim_datetime[['datetime_id']], on='datetime_id', how='left')



# 只保留 fact 表需要的 id 字段和指标字段
fact_jobs = fact_jobs[[
    'id','title_id', 'company_id', 'location_id', 'job_type_id', 'industry_id',
    'salary_id', 'accredited_id', 'datetime_id', 'active_id'
    
]]

print(fact_jobs.head(20))

       id  title_id  company_id  location_id  job_type_id  industry_id  salary_id  accredited_id  datetime_id  active_id
0   21070         1           1            1            1            1          1              1     20240824          0
1   21840         2           2            1            1            2          1              1     20240824          0
2   22213         3           2            1            1            2          1              1     20240824          0
3   26397         4           3            1            1            3          2              1     20240824          0
4   27671         5           4            2            1            4          1              1     20240824          0
5   27946         6           5            3            1            4          1              2     20240824          0
6   29815         7           6            1            1            5          1              1     20240824          0
7   29830         8           6 

In [76]:
fact_jobs.to_csv('tables/fact_jobs.csv', index=False)